# Détection automatique d’accidents anormaux à partir de données météorologiques, temporelles et spatiales

Ce notebook présente une analyse exploratoire et une détection d'anomalies sur les accidents de la route aux États-Unis, en utilisant des données météorologiques, temporelles et spatiales. Nous utilisons Spark pour le traitement des données volumineuses, puis des techniques de machine learning pour la détection d'accidents atypiques.

## 1. Import des librairies et initialisation Spark

Nous importons les librairies nécessaires pour le traitement, la visualisation et le machine learning.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import hour, dayofweek, month, col, when
from sklearn.preprocessing import StandardScaler
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from sklearn.ensemble import IsolationForest
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession.builder\
    .master("local[*]")\
    .appName("Accident")\
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "4g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

## 2. Chargement des données

Lecture des données d'accidents depuis un fichier Parquet prétraité.

In [ ]:
data = spark.read.parquet("../../data/processed_accidents.parquet")
data.show(5, truncate=False)

## 3. Prétraitement des données

Affichage du schéma et sélection des colonnes pertinentes pour l'analyse.

In [ ]:
data.printSchema()

In [ ]:
colonnes_utiles = [
    "Severity", "Start_Time", "Distance(mi)", "Temperature(F)", "Humidity(%)",
    "Wind_Speed(mph)", "Visibility(mi)", "Weather_Condition", "Sunrise_Sunset", "Start_Lat", "Start_Lng"]
data_reduit = data.select(*colonnes_utiles)
data_sample = data_reduit.sample(fraction=0.1, seed=42)
df = data_sample.toPandas()

## 4. Analyse exploratoire et visualisations

Quelques visualisations pour mieux comprendre la distribution des accidents selon la gravité, le temps, la météo et la localisation.

In [ ]:
import plotly.express as px

# Échantillonnage si dataset très grand
sample_df = df[["Start_Lat", "Start_Lng", "Severity"]].dropna().sample(n=10000, random_state=42)

fig = px.scatter_mapbox(
    sample_df,
    lat="Start_Lat",
    lon="Start_Lng",
    color="Severity",
    color_continuous_scale="OrRd",
    zoom=3,
    height=600,
    title="Accidents de la route par gravité - USA"
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Échantillonnage si dataset volumineux
sample_df = df[["Start_Lat", "Start_Lng", "Severity"]].dropna().sample(n=5000, random_state=42)

# Création d'une carte centrée sur les US
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4, tiles="CartoDB positron")

# Ajout de clusters
marker_cluster = MarkerCluster().add_to(m)

# Ajout des marqueurs
for _, row in sample_df.iterrows():
    folium.Marker(
        location=[row["Start_Lat"], row["Start_Lng"]],
        popup=f"Gravité : {row['Severity']}",
        icon=folium.Icon(color="red" if row["Severity"] >= 3 else "orange")
    ).add_to(marker_cluster)

# Affichage de la carte
m


In [ ]:
import plotly.express as px
fig = px.histogram(df, x='Severity', nbins=4, title='Distribution de la gravité des accidents (Severity)',
                   labels={'Severity': 'Gravité (1=faible, 4=grave)'})
fig.show()

In [ ]:
# 3. Accidents par jour de la semaine
df['DayOfWeek'] = pd.to_datetime(df['Start_Time']).dt.dayofweek
jours = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']
plt.figure(figsize=(10, 5))
sns.countplot(x=df['DayOfWeek'], palette='Set2')
plt.xticks(ticks=range(7), labels=jours)
plt.title("Répartition des accidents par jour de la semaine")
plt.xlabel("Jour")
plt.ylabel("Nombre d'accidents")
plt.tight_layout()
plt.show()

In [ ]:
# 4. Conditions météo
top_weather = df['Weather_Condition'].value_counts().nlargest(10)
plt.figure(figsize=(10, 5))
sns.barplot(x=top_weather.values, y=top_weather.index, palette="coolwarm")
plt.title("Top 10 des conditions météo lors des accidents")
plt.xlabel("Nombre d'accidents")
plt.ylabel("Condition météo")
plt.tight_layout()
plt.show()

In [ ]:
# 5. Température vs gravité
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='Severity', y='Temperature(F)', palette='pastel')
plt.title("Distribution des températures selon la gravité des accidents")
plt.xlabel("Gravité")
plt.ylabel("Température (°F)")
plt.tight_layout()
plt.show()

### Corrélations entre variables numériques

Analyse de la corrélation entre la gravité des accidents et les autres variables numériques.

In [ ]:
numerical_cols = ['Severity', 'Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
                  'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']
df_corr = data.select(numerical_cols).toPandas()
corr_matrix = df_corr[numerical_cols].corr()
plt.figure(figsize=(8, 6))
sns.barplot(x=corr_matrix['Severity'].drop('Severity').abs().sort_values(ascending=False),
            y=corr_matrix['Severity'].drop('Severity').abs().sort_values(ascending=False).index)
plt.title("Corrélation absolue avec la gravité des accidents (Severity)")
plt.xlabel("Corrélation absolue")
plt.tight_layout()
plt.show()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation – Variables numériques")
plt.tight_layout()
plt.show()

## 5. Détection d'anomalies avec Isolation Forest

Nous utilisons l'algorithme Isolation Forest pour détecter les accidents atypiques en fonction de variables météorologiques, temporelles et spatiales.

In [ ]:
df_sample = data.sample(fraction=0.5, seed=42).toPandas()
features = [
    'Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
    'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)',
    'Sunrise_Sunset_indexed', 'Civil_Twilight_indexed', 'Nautical_Twilight_indexed',
    'Astronomical_Twilight_indexed', 'Weather_Condition_indexed',
    'Wind_Direction_indexed', 'State_indexed', 'City_indexed'
]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_sample[features])
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
df_sample['anomaly_score'] = model.fit_predict(X_scaled)
df_sample['is_anomaly'] = (df_sample['anomaly_score'] == -1).astype(int)
print(df_sample['is_anomaly'].value_counts())


In [ ]:
df_sample['Start_Time'] = pd.to_datetime(df_sample['Start_Time'])
df_sample['hour'] = df_sample['Start_Time'].dt.hour
plt.figure(figsize=(8, 4))
sns.countplot(data=df_sample, x='hour', hue='is_anomaly')
plt.title("Anomalies détectées par heure")
plt.xlabel("Heure")
plt.ylabel("Nombre d'accidents")
plt.legend(title="Anomalie")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data=df_sample, x='is_anomaly', y='Severity')
plt.title("Gravité (Severity) selon anomalie détectée")
plt.xlabel("Anomalie (1=oui)")
plt.ylabel("Gravité")
plt.tight_layout()
plt.show()

### Interprétation des résultats

- La distribution des anomalies détectées par heure permet d'identifier des périodes atypiques.
- La gravité des accidents détectés comme anomalies est comparée à celle des cas normaux pour mieux comprendre leur impact.